### Sentiment Analysis either posititve or negative
#### We use here word-wise computation using the concept of bucketing

In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import datetime
import spacy
import sklearn

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
# f = open('../../data/sentiment/positive')
# pos = f.read()
# f.close()

# f = open('../../data/sentiment/negative')
# neg = f.read()
# f.close()
spacy.load

<function spacy.load>

In [7]:
# Load Spacy word embeddings
#word_embeddings = spacy.load('en', vectors='glove.6B.50d.txt')
word_embeddings =spacy.load('en_core_web_md')
print word_embeddings
q= word_embeddings(u'laptop')
print q.vector.shape

(300,)


 ### Theory fot this block
 
 Okay. So say your sequence has 16 words, but your bucket is 10-20 words. In seq_to_data you want to make your sequence’s length equal to 20. So you do padding for the remaining 4 words.


In seq_data_matrix, you bucketing the sequences and are concatenating same-length sequences into one matrix for each bucket.


It had 6 words and you added padding for the remains 94 words.

Yes. So for 0 to 5, you added word embeddings from your sequence. Remaining are zero because data_mat = np.zeros()

 Okay. For a moment, forget that data_mat has shape 1,100,300. Say it’s shape is 100,300.
 Now if I perform the same operation, data_mat’s first row will be substituted by the first row of data.
 Since data also has 300 columns, it will fit easily.

In [38]:
# Create a function to get vector format data for a sequence
#for one batch computation
def sequence_to_data(seq, max_len=None):
    seq = unicode(seq)
    #print seq
    #print seq.split()        #[u'hello!', u'what', u'is', u'the', u'date', u'today?']
    data = [word_embeddings(ix).vector for ix in seq.split()] #Split the sequence on the basis of words and 
    #take vector of those words from word_embeddings glove now data have vectors of words
    #print len(data)         # data contain 6 length vector of 6 words
    #print data[3].shape   #(300,)         # 6 because here we do word wise computation
    if max_len is None:
        max_len = len(data)
    
    data_mat = np.zeros((1, max_len, 300))  #(1,100,300) matrix of size this (1=batch,100 seq lenght and 300 features )
    #print min(len(data),max_len)       # 6
    for ix in range(min(len(data), max_len)):  #Here concept of bucketing we take max_len of bucket 
        data_mat[:, ix, :] = data[ix]
        #print data_mat.shape                  #(1, 100, 300)
    
    return data_mat

#For multiple batch consists of varying length sequence
def seq_data_matrix(seq_data, max_len=None):          #This fucntion is for processing multibatch at atime means in a multiple batch have multipl e length sequences
    n_seq = len(seq_data)
    #print n_seq
    data = np.concatenate([sequence_to_data(ix, max_len) for ix in seq_data], axis=0)
    #print data.shape
    return data
    
q = sequence_to_data(u'hello! what is the date today?', 100)
print q.shape
#m= seq_data_matrix(u'hello! what is the date today?',100)
#print m.shape

(1, 100, 300)


In [ ]:
# df = pd.DataFrame([], columns=['text', 'score'])
# for ix in pos.split('\n'):
#     text = ix.strip().lower()
#     if len(text) > 1:
#         df = df.append({'text': text, 'score': 1}, ignore_index=True)
#     # print sequence_to_data(ix.strip().lower()).shape

# for ix in neg.split('\n'):
#     text = ix.strip().lower()
#     if len(text) > 1:
#         df = df.append({'text': text, 'score': 0}, ignore_index=True)

In [40]:
#df = sklearn.utils.shuffle(df).reset_index(drop=True) 

#This is required because we build a dataframe own #
#its own in which at top enteries will be of poditive and botton enteries of sentiments are -ve means class (0) 
#so if we didnt shuffle then our model most of time gives the same ansuwer that is +ve because it becme
#familiar with +v eneteries that why we going to shuffle them


df = pd.read_csv('/home/chintan/Class_Code/Class17(LSTMS continue)/imdb_tr.csv')#, sep='|')#, index_col=0)

In [11]:
df.head()
df


,row_Number,text,polarity
0,2148,"first think another Disney movie, might good, ...",1
1,23577,"Put aside Dr. House repeat missed, Desperate H...",0
2,1319,"big fan Stephen King's work, film made even gr...",1
3,13358,watched horrid thing TV. Needless say one movi...,0
4,9495,truly enjoyed film. acting terrific plot. Jeff...,1
5,2154,"memory ""The Last Hunt"" stuck since saw 1956 13...",1
6,19880,"Shakespeare fan, appreciate Ken Branagh done b...",0
7,2073,privilege watching Scarface big screen beautif...,1
8,12001,real classic. shipload sailors trying get town...,1
9,9373,Serials short subjects originally shown theate...,1


### Just an example to understand the concept of .apply function

In [12]:
a = pd.DataFrame([], columns=['x'])

for ix in range(100):
    a = a.append({'x': ix}, ignore_index=True)
#print a

In [13]:
a['x_sq'] = a.x.apply(lambda x: x**2)
a.head()

,x,x_sq
0,0,0
1,1,1
2,2,4
3,3,9
4,4,16


In [14]:
## Original code

In [15]:
#Drop a column
col_to_drop=['row_Number']
df=df.drop(col_to_drop,axis=1)



In [16]:
df.head()

,text,polarity
0,"first think another Disney movie, might good, ...",1
1,"Put aside Dr. House repeat missed, Desperate H...",0
2,"big fan Stephen King's work, film made even gr...",1
3,watched horrid thing TV. Needless say one movi...,0
4,truly enjoyed film. acting terrific plot. Jeff...,1


In [17]:
#df= pd.DataFrame(polarity ,dtype='float')

In [18]:
df['len'] = df['text'].str.split().apply(lambda x: len(x))
# df = df.sort_index(ascending=False).reset_index(drop=True)

In [19]:
df.head()
df


,text,polarity,len
0,"first think another Disney movie, might good, ...",1,52
1,"Put aside Dr. House repeat missed, Desperate H...",0,86
2,"big fan Stephen King's work, film made even gr...",1,193
3,watched horrid thing TV. Needless say one movi...,0,63
4,truly enjoyed film. acting terrific plot. Jeff...,1,65
5,"memory ""The Last Hunt"" stuck since saw 1956 13...",1,43
6,"Shakespeare fan, appreciate Ken Branagh done b...",0,41
7,privilege watching Scarface big screen beautif...,1,331
8,real classic. shipload sailors trying get town...,1,25
9,Serials short subjects originally shown theate...,1,124


In [36]:
data=df
data.text= data.text.decode('utf-8')
# df.to_csv('../../data/sentiment/dataset.csv', sep='|')

AttributeError: 'Series' object has no attribute 'decode'

In [21]:
bucket_sizes = [[0, 30], [30, 60], [60, 90], [90, 120], [120, 200],[200,250]] # we make different bucket sizes based on analysis of data
#print len(bucket_sizes)  #5
#for i in bucket_sizes:
#    print i[1]

#This function is useed to assign the bucket indices for each sequence length
def assign_bucket(x):   
    for bucket in bucket_sizes:                       #One by one access all buckets
        if x > bucket[0] and x <= bucket[1]:          #based on given length and this condtion 
            return bucket_sizes.index(bucket)         #Succesfully returs the bucket index at which a=the seq length falls
    return len(bucket_sizes)-1

In [22]:
#x= assign_bucket()

In [23]:
df['bucket'] = df.len.apply(assign_bucket)
df.head()

,text,polarity,len,bucket
0,"first think another Disney movie, might good, ...",1,52,1
1,"Put aside Dr. House repeat missed, Desperate H...",0,86,2
2,"big fan Stephen King's work, film made even gr...",1,193,4
3,watched horrid thing TV. Needless say one movi...,0,63,2
4,truly enjoyed film. acting terrific plot. Jeff...,1,65,2


In [24]:
df = df.sort_values(by=['bucket'])
df.head()

,text,polarity,len,bucket
18991,anybody succeed getting movie?<br /><br />It's...,0,27,0
18839,"definite Lars von Trier Movie, favorite, rank ...",1,23,0
2623,attractive capable cast lost deadly boring reh...,0,23,0
8111,"disappointed movie like french actors liked ""B...",0,20,0
18792,"I've seen 820 movies released 1931-39, INFORME...",0,27,0


In [25]:
df[(df.bucket == 0)]      # For only those sequences which fall in bucket 0 means whose seq length is [0,10]

,text,polarity,len,bucket
18991,anybody succeed getting movie?<br /><br />It's...,0,27,0
18839,"definite Lars von Trier Movie, favorite, rank ...",1,23,0
2623,attractive capable cast lost deadly boring reh...,0,23,0
8111,"disappointed movie like french actors liked ""B...",0,20,0
18792,"I've seen 820 movies released 1931-39, INFORME...",0,27,0
18778,"Grey Gardens shocking, amusing, sad mesmerizin...",1,27,0
8187,"Ok,so.....guy gets bitten bat turns bat (well,...",0,24,0
18704,rated 3. dubbing bad seen. plot - yuck. I'm su...,0,29,0
18683,"one favourite movies time. dialogue, constant ...",1,23,0
8214,remember movie 50s college. one funniest satir...,1,20,0


### Now make the batch

In [26]:
def make_batch(data, batch_size=10, gpu=True):  #data= df
    for bx in range(len(bucket_sizes)):
        bucket_data = df[(df.bucket == bx)].reset_index(drop=True)
        # print bx, bucket_sizes[bx][1], bucket_data.shape
        
        start = 0
        stop = start + batch_size
        
        while start < bucket_data.shape[0]:
            seq_length = bucket_sizes[bx][1]
            section = bucket_data[start:stop]
            X_data = seq_data_matrix(section.text, max_len=seq_length)
            y_data = section.score
            
            if gpu:
                yield Variable(torch.FloatTensor(X_data), requires_grad=True), Variable(torch.LongTensor(y_data))#.cuda()
            else:
                yield Variable(torch.FloatTensor(X_data), requires_grad=True), Variable(torch.LongTensor(y_data))
            
            start = stop
            stop = start + batch_size

In [27]:
for ix, iy in make_batch(df, batch_size=1000):
    print ix.shape, iy.shape

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 150: ordinal not in range(128)

In [16]:
# df.head(10)
# Printing colored text (Useful later)
# print colored("hello red world", 'blue')# print 'a'

In [26]:
class SeqModel(nn.Module):
    def __init__(self, in_shape=None, out_shape=None, hidden_shape=None):
        super(SeqModel, self).__init__()
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.hidden_shape = hidden_shape
        self.n_layers = 1
        
        self.rnn = nn.LSTM(
            input_size=self.in_shape,
            hidden_size=self.hidden_shape,
            num_layers=self.n_layers,
            batch_first=True
        )
        self.lin = nn.Linear(self.hidden_shape, 64)
        self.dropout = nn.Dropout(0.42)
        self.out = nn.Linear(64, self.out_shape)
    
    def forward(self, x, h):
        r_out, h_state = self.rnn(x, h)
        last_out = r_out[:, -1, :]
        y = F.tanh(self.lin(last_out))
        y = self.dropout(y)
        y = F.softmax(self.out(y))
        return y
    
    def predict(self, x):
        h_state = self.init_hidden(1, gpu=False)
        
        x = sequence_to_data(x)
        pred = self.forward(torch.FloatTensor(x), h_state)
        
        return pred
    
    def get_embedding(self, x):
        h_state = self.init_hidden(1, gpu=False)
        
        x = sequence_to_data(x)
        r_out, h = self.rnn(torch.FloatTensor(x), h_state)
        last_out = r_out[:, -1, :]
        
        return last_out.data.numpy()
            
    def init_hidden(self, batch_size, gpu=True):
        if gpu:
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape).cuda()),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)).cuda())
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)))

In [48]:
model = SeqModel(in_shape=300, hidden_shape=256, out_shape=2)

print model
model.cuda()

SeqModel(
  (rnn): LSTM(300, 256, batch_first=True)
  (lin): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.42)
  (out): Linear(in_features=64, out_features=2, bias=True)
)


SeqModel(
  (rnn): LSTM(300, 256, batch_first=True)
  (lin): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.42)
  (out): Linear(in_features=64, out_features=2, bias=True)
)

In [49]:
# model.predict('hello bad world')

# Load the model
model.load_state_dict(torch.load('/home/shubham/all_projects/CB/Summer_2018/data/checkpoints/seq_lstm_bucket/model_256h_epoch_700.ckpt'))

In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
criterion = nn.CrossEntropyLoss()

In [35]:
# Set to train mode
# model.cuda()
model.train()

for epoch in range(50):
    total_loss = 0
    N = 0
    for step, (b_x, b_y) in enumerate(make_batch(df, batch_size=200)):
        # print step, b_x.shape, b_y.shape
        bsize = b_x.size(0)
        
        h_state = model.init_hidden(bsize, gpu=True)

        pred = model(b_x, h_state)
        loss = criterion(pred, b_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
        N += 1.0
        if step%20 == 0:
            print 'Loss: {} at Epoch: {} | Step: {}'.format(loss, epoch, step)
        
    print "Overall Average Loss: {} at Epoch: {}".format(total_loss / float(N), epoch)
    
    # Save model checkpoints
    if epoch % 10 == 0:
        torch.save(model.state_dict(), "/home/shubham/all_projects/CB/Summer_2018/data/checkpoints/seq_lstm_bucket/model_256h_epoch_{}.ckpt".format(epoch))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 0.69742333889 at Epoch: 0 | Step: 0
Loss: 0.69163531065 at Epoch: 0 | Step: 20
Loss: 0.692053377628 at Epoch: 0 | Step: 40
Overall Average Loss: 0.694422006607 at Epoch: 0
Loss: 0.692090451717 at Epoch: 1 | Step: 0
Loss: 0.687447428703 at Epoch: 1 | Step: 20
Loss: 0.689199209213 at Epoch: 1 | Step: 40
Overall Average Loss: 0.689712703228 at Epoch: 1
Loss: 0.681926727295 at Epoch: 2 | Step: 0
Loss: 0.668727934361 at Epoch: 2 | Step: 20
Loss: 0.647350132465 at Epoch: 2 | Step: 40
Overall Average Loss: 0.674889683723 at Epoch: 2
Loss: 0.63601154089 at Epoch: 3 | Step: 0
Loss: 0.608234405518 at Epoch: 3 | Step: 20
Loss: 0.593071639538 at Epoch: 3 | Step: 40
Overall Average Loss: 0.638657689095 at Epoch: 3
Loss: 0.577009916306 at Epoch: 4 | Step: 0
Loss: 0.578892111778 at Epoch: 4 | Step: 20
Loss: 0.581489622593 at Epoch: 4 | Step: 40
Overall Average Loss: 0.612688839436 at Epoch: 4
Loss: 0.549013614655 at Epoch: 5 | Step: 0
Loss: 0.570038259029 at Epoch: 5 | Step: 20
Loss: 0.57555544

In [50]:
# Make ppredictions
model.eval()
model.cpu()

SeqModel(
  (rnn): LSTM(300, 256, batch_first=True)
  (lin): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.42)
  (out): Linear(in_features=64, out_features=2, bias=True)
)

In [51]:
model.predict('I am going to some place to take play a game')

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[ 1.0000e+00,  1.3468e-15]])

In [29]:
import sklearn.metrics

In [30]:
model.cpu()

v1 = model.get_embedding('I am going to a place')
v2 = model.get_embedding('I am not going')
print v1.shape, v2.shape

(1, 256) (1, 256)


In [16]:
sklearn.metrics.pairwise.cosine_distances(v1, v2)

AttributeError: 'module' object has no attribute 'metrics'

#### For self understanding for upper concepts 

In [89]:
m= pd.DataFrame([],columns=['x','Abc','Ci'])
#print m
for ix in range(100):
    #m=m.append({'x':ix},{'Abc':ix**2},ignore_index=True)  #error append() got multiple values for keyword argument 'ignore_index'
    m=m.append({'x':ix},ignore_index=True)

In [90]:
m.head()

,x,Abc,Ci
0,0.0,NaN,NaN
1,1.0,NaN,NaN
2,2.0,NaN,NaN
3,3.0,NaN,NaN
4,4.0,NaN,NaN


In [93]:
m['x_square']= a.x.apply(lambda x:x**2)
m['AbcPlus']= a.x.apply(lambda NaN: NaN+12)

In [94]:
m.head()

,x,Abc,Ci,x_square,AbcPlus
0,0.0,NaN,NaN,0,12
1,1.0,NaN,NaN,1,13
2,2.0,NaN,NaN,4,14
3,3.0,NaN,NaN,9,15
4,4.0,NaN,NaN,16,16


In [153]:
str='This is  Chintan jain and i want to take a job in this 6 months only please help me '
m= str.split()
print m
print len(m)

['This', 'is', 'Chintan', 'jain', 'and', 'i', 'want', 'to', 'take', 'a', 'job', 'in', 'this', '6', 'months', 'only', 'please', 'help', 'me']
19


In [46]:
a= np.array([[1,2,3],[4,5,6],[7,8,9]])
print a.shape
print a[0][1]

(3, 3)
2
